Using https://learnbybuilding.ai/tutorials/rag-from-scratch as a tutorial

This ends the tutorial at https://learnbybuilding.ai/tutorials/rag-from-scratch 
Now to go into https://learnbybuilding.ai/tutorials/rag-from-scratch-part-2-semantics-and-cosine-similarity

## Trying to isolate embedding before putting into Weaviate

In [ ]:
pip install -r ../requirements.in

In [ ]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Title, NarrativeText, Text
from unstructured.chunking.basic import chunk_elements
from typing import List
import os
import weaviate
from weaviate.util import generate_uuid5
import ollama

In [ ]:
../setup.sh

In [ ]:
import ollama
ollama.pull("mxbai-embed-large:v1")
ollama.list()

In [ ]:
#FILE_PATH = "../docs/NIST.SP.800-171r2.pdf"
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH)

In [ ]:
titles = [elem for elem in elements if elem.category == "Title"]

for title in titles:
    print(title.text)

In [ ]:
import textwrap

narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]

for index, elem in enumerate(narrative_texts[:5]):
    print(f"Narrative text {index + 1}:")
    print("\n".join(textwrap.wrap(elem.text, width=70)))
    print("\n" + "-" * 70 + "\n")

In [ ]:
for text in narrative_texts:
    print(text.text)

In [ ]:
client = weaviate.connect_to_local()

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large:v1",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)


In [ ]:

pass_embedding = ollama_embedding.get_text_embedding_batch(
    [narrative_texts[1],narrative_texts[50]], show_progress=True
)
print(pass_embedding)


In [ ]:

query_embedding = ollama_embedding.get_query_embedding("What is a paladin?")
print(query_embedding)

In [ ]:
data_objects = []
# Assuming you have extracted abstracts from the PDFs
for pdf_file in list_of_pdf_files:
    abstract = extract_abstract(pdf_file)
    data_object = {"source": pdf_file.name, "abstract": abstract}
    data_objects.append(data_object)

# Import the objects into Weaviate
client.batch.configure(batch_size=100)
with client.batch as batch:
    for data_object in data_objects:
        batch.add_object(data_object)


In [ ]:
for i, d in enumerate(elements):
  response = ollama.embeddings(model="mxbai-embed-large:v1", prompt=d)

https://ollama.com/blog/embedding-models

In [ ]:
pip install ollama chromadb


In [ ]:
import ollama
import chromadb


In [ ]:
client.delete_collection(name="docs")

In [ ]:

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model="mxbai-embed-large:v1", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [ ]:
# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large:v1"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="mistral",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

Now that we've proved that this actually works, let's try to use OUR data

In [2]:
client = weaviate.connect_to_local()


NameError: name 'weaviate' is not defined

In [1]:
client.collections.delete(name="PHB")

NameError: name 'client' is not defined

In [ ]:
collection = client.collections.create(name="PHB")

In [ ]:
narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]
titles = [elem for elem in elements if elem.category == "Title"]

for index, elem in enumerate(narrative_texts[:5]):
    response = ollama.embeddings(model="mxbai-embed-large:v1", prompt=elem)
    embedding = response["embedding"]
    print(".", end="", flush=True)
    
    
    